# Bookish: Segundo Paso
Aqui vamos acotar nuestro dataset para solo tener los primeros 1000 libros. Estos libros son los que mas tuvieron una calificacion 

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
bookDownloaded = drive.CreateFile({'id':'18wJwL7EqyYQ-sWe0LKGZKVUyCIzUC_xm'})
userDownloaded = drive.CreateFile({'id':'1KJkNATp72B8HaqhsgS1wsgLIdXnDdsSi'})
ratingDownloaded = drive.CreateFile({'id':'1UN6dXHIjsiPjAgmiEdOKoSMJb1fQxMu5'})

In [ ]:
bookDownloaded.GetContentFile('books.csv')
userDownloaded.GetContentFile('users.csv')
ratingDownloaded.GetContentFile('ratings.csv')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')
import os, sys
import re

pd.set_option('display.max_colwidth', -1)

In [ ]:
books = pd.read_csv('books.csv')
books.head()

,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
users = pd.read_csv('users.csv')
users.head()

,Unnamed: 0,User-ID,Location,Age
0,0,1,"nyc, new york, usa",34
1,1,2,"stockton, california, usa",18
2,2,3,"moscow, yukon territory, russia",34
3,3,4,"porto, v.n.gaia, portugal",17
4,4,5,"farnborough, hants, united kingdom",34


In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating
0,0,276725,034545104X,0
1,1,276726,0155061224,5
2,2,276727,0446520802,0
3,3,276729,052165615X,3
4,4,276729,0521795028,6


In [ ]:
#El conjunto de datos de calificaciones debe tener solo los libros que existen en nuestro conjunto de datos de libros
ratings_new = ratings[ratings.ISBN.isin(books.ISBN)]

ratings.shape,ratings_new.shape

((1149780, 4), (1031136, 4))

In [ ]:
#Separamos las calificaciones implicitas y explicitas
ratings_explicit = ratings_new[ratings_new['Book-Rating'] != 0]
ratings_implicit = ratings_new[ratings_new['Book-Rating'] == 0]

In [ ]:
print(ratings_new.shape)
print(ratings_explicit.shape)
print(ratings_implicit.shape)
plt.style.use('fivethirtyeight')
plt.figure(figsize=(12, 8))
#sns.countplot(data=ratings_explicit , x='Book-Rating', palette='rocket_r')

(1031136, 4)
(383842, 4)
(647294, 4)


<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

# Top 1000 Books

In [ ]:
#sacamos el promedio de los libros
ratings_count_prom = pd.DataFrame(ratings_explicit.groupby(['ISBN'])['Book-Rating'].mean())
print(ratings_count_prom)

            Book-Rating
ISBN                   
0000913154  8.0        
0001046438  9.0        
000104687X  6.0        
0001047213  9.0        
0001047973  9.0        
...         ...        
B0001FZGPI  7.0        
B0001FZGRQ  9.0        
B0001GMSV2  8.0        
B0001I1KOG  10.0       
B000234N3A  9.0        

[149836 rows x 1 columns]


In [ ]:
ratings_count = pd.DataFrame(ratings_explicit.groupby(['ISBN'])['Book-Rating'].sum())
top1000 = ratings_count.sort_values('Book-Rating', ascending = False).head(1000)

In [ ]:
#hacemos un merge con los libros y sus ratings
top1000.merge(ratings_count_prom, left_index = True, right_on = 'ISBN').merge(books, left_index = True, right_on = 'ISBN')

,Book-Rating_x,Book-Rating_y,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
408,5787,8.185290,408,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
748,4108,8.435318,748,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday
522,3134,8.182768,522,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
2143,2798,8.939297,2143,059035342X,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),J. K. Rowling,1999,Arthur A. Levine Books
356,2595,8.452769,356,0142001740,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books
...,...,...,...,...,...,...,...,...
5255,250,8.333333,5255,0679732411,Gift from the Sea,Anne Morrow Lindbergh,1991,Pantheon Books
21201,250,8.064516,21201,0312978383,Winter Solstice,Rosamunde Pilcher,2001,St. Martin's Paperbacks
1431,250,8.333333,1431,0345325818,The Silmarillion,J.R.R. TOLKIEN,1985,Del Rey
15967,250,7.812500,15967,0345447867,The Apprentice,Tess Gerritsen,2003,Ballantine Books


In [ ]:
top1000.head()

,Book-Rating
ISBN,
0316666343,5787
0385504209,4108
0312195516,3134
059035342X,2798
0142001740,2595


In [ ]:
new_matrix = top1000.merge(ratings_explicit, on = 'ISBN' )
#Join de bd, una union entre dos dataframes; interseccion con el isbn; ratings_explicit tu tabla con datos crudos 

In [ ]:
new_matrix

,ISBN,Book-Rating_x,Unnamed: 0,User-ID,Book-Rating_y
0,0316666343,5787,1965,277439,7
1,0316666343,5787,2857,277752,7
2,0316666343,5787,9205,278633,9
3,0316666343,5787,11078,638,10
4,0316666343,5787,11316,709,3
...,...,...,...,...,...
67868,0452266564,250,1035666,247447,9
67869,0452266564,250,1052068,251422,4
67870,0452266564,250,1080228,258329,1
67871,0452266564,250,1093297,262317,6


In [ ]:
type(top1000)

pandas.core.frame.DataFrame

In [ ]:
# total de Ususarios que dieron una calificacion
ratings_count_users = pd.DataFrame(new_matrix['Book-Rating_x']>0).count()
print(ratings_count_users)

Book-Rating_x    67873
dtype: int64


In [ ]:
#Generating ratings matrix from explicit ratings table
top_ratings_matrix = new_matrix.pivot(index='User-ID', columns='ISBN', values='Book-Rating_y')
userID = top_ratings_matrix.index
ISBN = top_ratings_matrix.columns
print(top_ratings_matrix.shape)
top_ratings_matrix.head()

(25453, 1000)


ISBN,000649840X,0020442203,002542730X,0028604199,006000438X,0060008032,0060096195,006016848X,0060173289,0060175400,0060175966,0060188731,006019491X,0060199652,0060248025,0060391626,0060392452,0060502258,0060512822,0060915544,0060916508,0060920084,0060921145,0060922532,0060926317,0060928336,0060929790,0060929871,006092988X,0060930187,0060930535,0060931418,0060932759,0060934417,0060938455,0060958022,0060959037,0060964049,0060976845,0060977493,...,0877017883,0887307876,0894803204,0894805770,0894808249,089480829X,0971880107,140003065X,1400031346,1400031354,1400031362,1400032717,1400034779,1551668998,1558531025,155874262X,1558743669,1558744150,1558744630,1558745157,1558745718,1559029838,1565122968,1573221937,1573225517,1573225789,1573227331,1573228214,1573229326,1573229571,1576737330,1592400876,1844262553,1857022424,1878424319,1931561648,3257229534,3404148665,3423202327,3492045170
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Los NaNs no se pueden usar para los algoritmos de entrenamiento, lo vamos a remplazar con 0, que indicara que no hay una calificacion
#cambiamos el tipo de dato a int
top_ratings_matrix.fillna(0, inplace = True)
top_ratings_matrix = top_ratings_matrix.astype(np.int32)

In [ ]:
#verificamos las primeras filas
top_ratings_matrix.head(5)

ISBN,000649840X,0020442203,002542730X,0028604199,006000438X,0060008032,0060096195,006016848X,0060173289,0060175400,0060175966,0060188731,006019491X,0060199652,0060248025,0060391626,0060392452,0060502258,0060512822,0060915544,0060916508,0060920084,0060921145,0060922532,0060926317,0060928336,0060929790,0060929871,006092988X,0060930187,0060930535,0060931418,0060932759,0060934417,0060938455,0060958022,0060959037,0060964049,0060976845,0060977493,...,0877017883,0887307876,0894803204,0894805770,0894808249,089480829X,0971880107,140003065X,1400031346,1400031354,1400031362,1400032717,1400034779,1551668998,1558531025,155874262X,1558743669,1558744150,1558744630,1558745157,1558745718,1559029838,1565122968,1573221937,1573225517,1573225789,1573227331,1573228214,1573229326,1573229571,1576737330,1592400876,1844262553,1857022424,1878424319,1931561648,3257229534,3404148665,3423202327,3492045170
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
users_exp_ratings = users[users['User-ID'].isin(ratings_explicit['User-ID'])]
users_imp_ratings = users[users['User-ID'].isin(ratings_implicit['User-ID'])]

In [ ]:
print(users.shape)
print(users_exp_ratings.shape)
print(users_imp_ratings.shape)

(278858, 4)
(68091, 4)
(52451, 4)


In [ ]:
#checamos la escasez(sparsity)
sparsity=1.0-len(ratings_explicit)/float(users_exp_ratings.shape[0]*271360)
#n_books = 271360
print('The sparsity level of Book Crossing dataset is ' +  str(sparsity*100) + ' %')

The sparsity level of Book Crossing dataset is 99.99792261519545 %


In [ ]:
top_ratings_matrix.to_csv('tops.csv')

In [ ]:
books.loc[(books.ISBN == '0020442203'),:]

,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
9453,9453,0020442203,"Lion, the Witch and the Wardrobe",C.S. Lewis,1970,MacMillan Publishing Company.
